In [1]:
# Author: Michael Bramble | michael.s.bramble@jpl.nasa.gov
# Investigating EMIT L2B MIN products for acid mind drainage 
# 20240226 - v3 now includes both group_1 and group_2 minerals
# 20240330 - added exporting new array as netCDF file
# 20240411 - added exporting new array as geoTIFF file
# 20240429 - added quality check using band_depth band
# 20240430 - forked and cleaned version of min product

import os
import numpy as np
import math
import xarray as xr
import holoviews as hv
import hvplot.xarray
import netCDF4 as nc
import rasterio
import rioxarray
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import earthpy.plot as ep

# point to the EMITL2BMIN product
granule_asset_id_min = 'EMIT_L2B_MIN_001_20231015T215344_2328814_020.nc'
fp_min = f'/Users/bramble/Documents/emit/alum_gulch/{granule_asset_id_min}'

# point to the corresponding reflectance product
granule_asset_id_ref = 'EMIT_L2A_RFL_001_20231015T215344_2328814_020.nc'
fp_ref = f'/Users/bramble/Documents/emit/alum_gulch/{granule_asset_id_ref}'

# create asset ID for exported products
asset_id_min = '20231015T215344_2328814_020'

# import the emit_tools module
import sys
sys.path.append('/Users/bramble/My Drive/_JPL_AMD/EMIT-Data-Resources-main/python/modules/')
from emit_tools import emit_xarray
# help(emit_xarray)

# import min data using the emit tools
ds_geo_min = emit_xarray(fp_min, ortho=True)
# ds_geo_min

# import ref data using the emit tools
ds_geo_ref = emit_xarray(fp_ref, ortho=True)
# ds_geo_ref

# remove the bad bands
ds_geo_ref['reflectance'].data[:,:,ds_geo_ref['good_wavelengths'].data==0] = np.nan

In [2]:
# view the dataset to confirm contents
ds_geo_min

<xarray.Dataset>
Dimensions:             (latitude: 2009, longitude: 2107, mineral_name: 294)
Coordinates:
  * latitude            (latitude) float64 31.91 31.91 31.91 ... 30.82 30.82
  * longitude           (longitude) float64 -111.2 -111.2 ... -110.1 -110.1
    record              (mineral_name) float64 6.858e+03 744.0 ... 1.008e+03
  * mineral_name        (mineral_name) object 'Plastic_Tarp GDS339 Green W1R1...
    url                 (mineral_name) object 'https://crustal.usgs.gov/specl...
    group               (mineral_name) float64 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0
    library             (mineral_name) object 'splib06' 'sprlb06' ... 'sprlb06'
    index               (mineral_name) float64 1.0 2.0 3.0 ... 292.0 293.0 294.0
    elev                (latitude, longitude) float32 nan nan nan ... nan nan
    spatial_ref         int64 0
Data variables:
    group_1_band_depth  (latitude, longitude) float32 nan nan nan ... nan nan
    group_1_mineral_id  (latitude, longitude) float32 nan nan nan ... nan nan
    group_2_band_depth  (latitude, longitude) float32 nan nan nan ... nan nan
    group_2_mineral_id  (latitude, longitude) float32 nan nan nan ... nan nan
Attributes: (12/39)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-1.11249038e+02  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2B Estimated Mineral Identificat...
    granule_id:                        EMIT_L2B_MIN_001_20231015T215344_23288...
    Orthorectified:                    True

In [3]:
# crop the image to an area of interst

# # ALUM GULCH, LEAD QUEEN
max_lat = 31.55
min_lat = 31.45
max_lon = -110.8
min_lon = -110.7

# choose the group 1 minerals
band = ds_geo_min.group_1_mineral_id
band_depth = ds_geo_min.group_1_band_depth
# band

# select just the spatial subset around the mine
SliceData = band.sel({'latitude' : slice(max_lat, min_lat),
                      'longitude' : slice(max_lon, min_lon)})
SliceData_band_depth = band_depth.sel({'latitude' : slice(max_lat, min_lat),
                                       'longitude' : slice(max_lon, min_lon)})
# SliceData

# visualize the subset of data
SliceData.hvplot.image(cmap='viridis', aspect ='equal', rasterize=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'5fcec80a-a5f1-493d-afe4-245f5574a144': {'version…

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Callback.process_on_change() done, defined at /Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='p1010', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValue

In [4]:
# iterate through each pixel and make a new map classifiying the surface into the four 
# AMD minerals of interest
# 20240109 - update to include schwertmannite and copiapite, added full goethite list
# 20240129 - update to include the whole suite of jarosite spectra (high numbers in list)
# 20240129 - corrected some of the index numbers
# 20240226 - updated jarosite to include 9 and 49.
# 20240226 - swapped the i/j and lat/lon values. Matters for non-square data

band_lon =  SliceData['longitude']
band_lat =  SliceData['latitude']
idx_goethite = [3, 5, 6, 7, 8, 9, 18]
# {2: 'Goethite WS222 <250um MedGrn W1R1H_ AREF'}
# {3: 'Goethite0.02+Quartz GDS240 W1R1Ba'}
# {4: 'Goethite MPCMA2-B FineGr adj W1R1Bb'}
# {5: 'Goethite MPCMA2-C M-Crsgrad2 W1R1Hb AREF'}
# {6: 'Goethite WS222 coarse grain W1R1H_'}
# {7: 'Goethite_Thin_Film WS222 W1R1Ba'}
# {8: 'Goeth+qtz.5+Jarosite.5 AMX11 W1R1BbS'}
# {17: 'Goethite CU91-252 coatedchip W1R1H_'}
idx_hematite = [30, 31, 32, 33, 45, 47]
# {29: 'Hematite GDS27 W1R1Ha'}
# {30: 'Hematite_Thin_Film GDS27 W1R1Ba'}
# {31: 'Hematite FE2602 W1R1Bb'}
# {32: 'Hematite WS161 W1R1Hb'}
# {44: 'Nanohematite BR93-34B2 W1R1BbS'}
# {46: 'Nanohematite FBR93-34B2b ed1 W1R1Hb'}
idx_jarosite = [9, 15, 49, 204, 205, 206]
# {8: 'Goeth+qtz.5+Jarosite.5 AMX11 W1R1BbS'}
# {14: 'Jarosite GDS99 K 200C Syn W1R1BaS'}
# {48: 'Jarosite_on_Qtzite BR93-34A2 W1R1Bb'}
# {203: 'Jarosite GDS24 Na W1R1Bb'}
# {204: 'Jarosite GDS99 K 200C Syn W1R1Ba'}
# {205: 'Jarosite SJ-1 H3O 10-20% W1R1Bb'}

newarray = xr.DataArray(None, SliceData.coords, SliceData.dims)
for i in np.arange(0,len(band_lat)-1):
    for j in np.arange(0,len(band_lon)-1):
        match = SliceData[i,j]
        # print(match.values)
        if match.values == 10: # {9: 'Pyrite LV95-6A Weath on Tail W1R1Bb'}
            newarray[i,j] = 1
        elif match.values in idx_jarosite:
            newarray[i,j] = 2
        elif match.values in idx_goethite:
            newarray[i,j] = 3
        elif match.values in idx_hematite:
            newarray[i,j] = 4
        elif match.values == 11: # {10: 'Schwertmannite BZ93-1 W1R1Bb'}
            newarray[i,j] = 5
        elif match.values == 67: # {66: 'Copiapite GDS21 W1R1Bb'}
            newarray[i,j] = 6
        else:
            newarray[i,j] = 0



In [5]:
# quality check by removing low band depth values

for i in np.arange(0,len(band_lat)-1):
    for j in np.arange(0,len(band_lon)-1):
        match = SliceData_band_depth[i,j]
        if match.values < 0.01: 
            newarray[i,j] = 0


In [6]:
# visualize the output
newarray.hvplot.image(cmap='colorblind', aspect = 'equal').opts(color_levels=7)

:Image   [longitude,latitude]   (value)

In [7]:
# generate reflectance image for final visualization

rgb1 = ds_geo_ref.sel(wavelengths=850, method='nearest')
rgb2 = rgb1.sel({'latitude' : slice(max_lat, min_lat), 'longitude' : slice(max_lon, min_lon)})
rgb2.hvplot.image(cmap='Viridis', geo=True)

# rgb.sel(wavelengths=[650, 560, 470], method='nearest').hvplot.image(cmap='viridis', aspect = 'equal', rasterize=True)


:Image   [longitude,latitude]   (reflectance)

In [8]:
# repeat process but now for group_2 minerals

# choose the group 2 minerals
band_group_2 = ds_geo_min.group_2_mineral_id
band_depth_group_2 = ds_geo_min.group_2_band_depth

# select just the spatial subset around the mine
slice_data_group_2 = band_group_2.sel({'latitude' : slice(max_lat, min_lat),
                      'longitude' : slice(max_lon, min_lon)})

slice_data_band_depth_group_2 = band_depth_group_2.sel({'latitude' : slice(max_lat, min_lat),
                                                        'longitude' : slice(max_lon, min_lon)})


slice_data_group_2.hvplot.image(cmap='viridis', aspect ='equal', rasterize=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e7b49fa3-35b3-477a-8b88-bd5157d7a1ed': {'version…

Task exception was never retrieved
future: <Task finished name='Task-12' coro=<Callback.process_on_change() done, defined at /Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='p1594', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValu

In [9]:
# repeat process but now for group_2 minerals
# iterate through each pixel and make a new map classifiying the surface into the four 
# AMD minerals of interest
# 20240226 - initial version for group 2 minerals
# 20240226 - swapped the i/j and lat/lon values

band_lon =  slice_data_group_2['longitude']
band_lat =  slice_data_group_2['latitude']
idx_goethite = [3, 5, 6, 7, 8, 9, 18]
# {2: 'Goethite WS222 <250um MedGrn W1R1H_ AREF'}
# {3: 'Goethite0.02+Quartz GDS240 W1R1Ba'}
# {4: 'Goethite MPCMA2-B FineGr adj W1R1Bb'}
# {5: 'Goethite MPCMA2-C M-Crsgrad2 W1R1Hb AREF'}
# {6: 'Goethite WS222 coarse grain W1R1H_'}
# {7: 'Goethite_Thin_Film WS222 W1R1Ba'}
# {8: 'Goeth+qtz.5+Jarosite.5 AMX11 W1R1BbS'}
# {17: 'Goethite CU91-252 coatedchip W1R1H_'}
idx_hematite = [30, 31, 32, 33, 45, 47]
# {29: 'Hematite GDS27 W1R1Ha'}
# {30: 'Hematite_Thin_Film GDS27 W1R1Ba'}
# {31: 'Hematite FE2602 W1R1Bb'}
# {32: 'Hematite WS161 W1R1Hb'}
# {44: 'Nanohematite BR93-34B2 W1R1BbS'}
# {46: 'Nanohematite FBR93-34B2b ed1 W1R1Hb'}
idx_jarosite = [9, 15, 49, 204, 205, 206]
# {8: 'Goeth+qtz.5+Jarosite.5 AMX11 W1R1BbS'}
# {14: 'Jarosite GDS99 K 200C Syn W1R1BaS'}
# {48: 'Jarosite_on_Qtzite BR93-34A2 W1R1Bb'}
# {203: 'Jarosite GDS24 Na W1R1Bb'}
# {204: 'Jarosite GDS99 K 200C Syn W1R1Ba'}
# {205: 'Jarosite SJ-1 H3O 10-20% W1R1Bb'}

newarray_group_2 = xr.DataArray(None, slice_data_group_2.coords, slice_data_group_2.dims)
# for i in np.arange(0,len(band_lon)-1):
#     for j in np.arange(0,len(band_lat)-1):
for i in np.arange(0,len(band_lat)-1):
    for j in np.arange(0,len(band_lon)-1):
        match = slice_data_group_2[i,j]
        # print(match.values)
        if match.values == 10: # {9: 'Pyrite LV95-6A Weath on Tail W1R1Bb'}
            newarray_group_2[i,j] = 1
        elif match.values in idx_jarosite:
            newarray_group_2[i,j] = 2
        elif match.values in idx_goethite:
            newarray_group_2[i,j] = 3
        elif match.values in idx_hematite:
            newarray_group_2[i,j] = 4
        elif match.values == 11: # {10: 'Schwertmannite BZ93-1 W1R1Bb'}
            newarray_group_2[i,j] = 5
        elif match.values == 67: # {66: 'Copiapite GDS21 W1R1Bb'}
            newarray_group_2[i,j] = 6
        else:
            newarray_group_2[i,j] = 0



In [11]:
# quality check by removing low band depth values

for i in np.arange(0,len(band_lat)-1):
    for j in np.arange(0,len(band_lon)-1):
        match = slice_data_band_depth_group_2[i,j]
        if match.values < 0.01: 
            newarray_group_2[i,j] = 0


In [12]:
# visualize the output
newarray_group_2.hvplot.image(cmap='colorblind', aspect = 'equal').opts(color_levels=7)

:Image   [longitude,latitude]   (value)

In [13]:
# triple product visulization

plot = (rgb2.hvplot.image(cmap='Viridis', geo=True, title=granule_asset_id_ref+"\nReflectance at 850 nm", width=600, height=600) + newarray.hvplot.image(cmap='colorblind', geo=True, title=granule_asset_id_min+"\nAMD-relevant mineral detections", width=600, height=600).opts(color_levels=7) + newarray_group_2.hvplot.image(cmap='colorblind', geo=True, title=granule_asset_id_min+"\nAMD-relevant mineral detections", width=600, height=600).opts(color_levels=7))
output_name = granule_asset_id_ref + '.html'
hvplot.save(plot, output_name)
plot


:Layout
   .Image.I   :Image   [longitude,latitude]   (reflectance)
   .Image.II  :Image   [longitude,latitude]   (value)
   .Image.III :Image   [longitude,latitude]   (value)

In [14]:
# export to netCDF file

# group 1 minerals
# remove "None" values for netCDF file
newarray_out = newarray
newarray_out = newarray_out.fillna(value=0)
# newarray_out

temp_filename_nc = fr'/Users/bramble/Documents/emit/alum_gulch/{asset_id_min}_amd_min_group_1.nc'
newarray_out.to_netcdf(temp_filename_nc)

# group 2 minerals
# remove "None" values for netCDF file
newarray_out_group_2 = newarray_group_2
newarray_out_group_2 = newarray_out.fillna(value=0)

temp_filename_nc = fr'/Users/bramble/Documents/emit/alum_gulch/{asset_id_min}_amd_min_group_2.nc'
newarray_out_group_2.to_netcdf(temp_filename_nc)


In [15]:
# export to geotiff file

ds = newarray_out.astype(np.float32)
ds.rio.write_crs("epsg:4326", inplace=True)
temp_filename_exp = fr'/Users/bramble/Documents/emit/alum_gulch/{asset_id_min}_amd_min_group_1.tif'
ds.rio.to_raster(temp_filename_exp)

ds = newarray_out_group_2.astype(np.float32)
ds.rio.write_crs("epsg:4326", inplace=True)
temp_filename_exp = fr'/Users/bramble/Documents/emit/alum_gulch/{asset_id_min}_amd_min_group_2.tif'
ds.rio.to_raster(temp_filename_exp)